### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [12]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
file_to_load  = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(file_to_load)

In [13]:
# Make sure every cell of the data is filled
purchase_data.count()
# Every column has 780 rows so there is no empty cells in the data

Purchase ID    780
SN             780
Age            780
Gender         780
Item ID        780
Item Name      780
Price          780
dtype: int64

In [14]:
# Notes: 1 Player can buy multiple games
players_df    = purchase_data['SN']
total_players = len(players_df.value_counts())

print("The total number of players is: " + str(total_players))

The total number of players is: 576


In [15]:
# * Number of Unique Items
unique_values = purchase_data['Item Name'].value_counts()
number_of_unique_items = len(unique_values)
# * Average Purchase Price
average_purchase_price = purchase_data['Price'].mean()
# * Total Number of Purchases
total_number_of_purchases = len(purchase_data['Purchase ID'])
# * Total Revenue
total_revenue = purchase_data['Price'].sum()

print("The     Total     Revenue      is: "    + str("%.2f" % total_revenue))
print("The  Total Number of Purchases is:  "   + str("%.2f" % total_number_of_purchases))
print("The  Number  of  Unique Items  is:  "   + str("%.2f" % number_of_unique_items)) 
print("The  Average  Purchase  Price  is:    " + str("%.2f" % average_purchase_price))


The     Total     Revenue      is: 2379.77
The  Total Number of Purchases is:  780.00
The  Number  of  Unique Items  is:  179.00
The  Average  Purchase  Price  is:    3.05


In [16]:
# Create a new dataframe with: ID  Gender  totalSpent
aggregation_functions = {'Price': 'sum', 'Gender': 'first', 'SN': 'count'}
filtered_df = purchase_data.groupby(purchase_data['SN']).aggregate(aggregation_functions)
genders_series = filtered_df['Gender'].value_counts()

# Divide the number of each gender by the total player for percentages
gender_percentages = round((genders_series / total_players) * 100,3)
print("The  male  % is: " + str(gender_percentages['Male'])   + '%')
print("The female % is: " + str(gender_percentages['Female']) + '%')
print("The Other / Non-Disclosed % is: " + str(gender_percentages['Other / Non-Disclosed']) + '%')
genders_series


The  male  % is: 84.028%
The female % is: 14.062%
The Other / Non-Disclosed % is: 1.91%


Male                     484
Female                    81
Other / Non-Disclosed     11
Name: Gender, dtype: int64

In [18]:
#   * The below each broken by gender
filtered_df["Total Games"] = players_df.value_counts()
#   * Purchase Count
aggregation_functions = {'Price': 'sum', 'Total Games': 'sum'}
purchase_count_by_gender = filtered_df.groupby(filtered_df['Gender']).aggregate(aggregation_functions)
#   * Average Purchase Total per Person by Gender
purchase_count_by_gender['Average Purchase'] = pd.Series({'Female':" %.2f" % (purchase_count_by_gender.at['Female', 'Price'] / genders_series['Female']),
                                                          'Male'  :" %.2f" % (purchase_count_by_gender.at['Male', 'Price']   / genders_series[ 'Male' ]),
                                                          'Other / Non-Disclosed':" %.2f" % (purchase_count_by_gender.at['Other / Non-Disclosed', 'Price'] / genders_series['Other / Non-Disclosed']),
                                                         })
#   * Average Purchase Price
filtered_df['Price'].mean()
#   * Total  Purchase  Value
total_purchase_value = purchase_count_by_gender['Price'].sum()

print("Purchase count by gender:\n",purchase_count_by_gender , "\n")
print("Average Purchase Price:  " + str(" %.2f" % filtered_df['Price'].mean()))
print("Total Purchase Value: "    + str(" %.2f" % total_purchase_value))

Purchase count by gender:
                          Price  Total Games Average Purchase
Gender                                                      
Female                  361.94          113             4.47
Male                   1967.64          652             4.07
Other / Non-Disclosed    50.19           15             4.56 

Average Purchase Price:   4.13
Total Purchase Value:  2379.77


In [7]:
### Age Demographics

#   * The below each broken into bins of 4 years (i.e. &lt;10, 10-14, 15-19, etc.)
max_age   = purchase_data['Age'].max() + 4
min_age   = purchase_data['Age'].min() - 4
age_range = [value for value in range(min_age,max_age,4)]
group_ranges = [f'{age_range[i-1]+1}-{age_range[i]+1}' for i in range(1,len(age_range))]
purchase_data["Age Range"] = pd.cut(purchase_data['Age'], age_range, labels=group_ranges)

#   * Purchase Count
#   * Average Purchase Price
#   * Total Purchase Value
#   * Average Purchase Total per Person by Age Group

aggregation_functions = {'Price': 'sum','Purchase ID': 'count','SN':'unique'}
age_demographics = purchase_data.groupby(purchase_data['Age Range']).aggregate(aggregation_functions)
age_demographics = age_demographics.rename(columns={'Price': 'Total Purchase Value','Purchase ID': 'Purchase Count','SN':'Total Players'})
age_demographics['Average Purchase Price'] = age_demographics['Total Purchase Value'] / age_demographics['Purchase Count']
age_demographics['Total Players'] = age_demographics['Total Players'].str.len()
age_demographics['Average Purchase per Person'] = age_demographics['Total Purchase Value']/age_demographics['Total Players']
age_demographics


,Total Purchase Value,Purchase Count,Total Players,Average Purchase Price,Average Purchase per Person
Age Range,,,,,
4-8,32.89,9,7,3.654444,4.698571
8-12,94.86,30,23,3.162000,4.124348
12-16,137.81,47,35,2.932128,3.937429
16-20,307.24,101,81,3.041980,3.793086
20-24,903.84,298,210,3.033020,4.304000
24-28,459.54,150,111,3.063600,4.140000
28-32,178.05,60,44,2.967500,4.046591
32-36,131.66,45,32,2.925778,4.114375
36-40,95.64,27,21,3.542222,4.554286


In [8]:
# ### Top Spenders

#   * Identify the the top 5 spenders in the game by total purchase value, then list (in a table):
#   * SN
#   * Purchase Count
#   * Average Purchase Price
#   * Total Purchase Value

top_spenders = filtered_df.sort_values(by='Price',ascending=False)
top_spenders = top_spenders.rename(columns={'Price': 'Total Purchase Value','Total Games': "Purchase Count"})
del top_spenders['SN']
del top_spenders['Gender']
top_spenders['Average Purchase Price'] =  (top_spenders['Total Purchase Value'] / top_spenders['Purchase Count'])
top_spenders.head(5)


,Total Purchase Value,Purchase Count,Average Purchase Price
SN,,,
Lisosia93,18.96,5,3.792000
Idastidru52,15.45,4,3.862500
Chamjask73,13.83,3,4.610000
Iral74,13.62,4,3.405000
Iskadarya95,13.10,3,4.366667


In [89]:
# ### Most Popular Items

# * Identify the 5 most popular items by purchase count, then list (in a table):
#   * Item ID
#   * Item Name
#   * Purchase Count
#   * Item Price
#   * Total Purchase Value

aggregation_functions = {'Purchase ID':'count','Item ID':'unique','Price':'sum'}
most_popular_items = purchase_data.groupby(purchase_data['Item Name']).aggregate(aggregation_functions)
most_popular_items = most_popular_items.rename(columns={'Purchase ID': 'Purchase Count','Price': 'Total Purchase Value'})
most_popular_items = most_popular_items.sort_values(by='Purchase Count',ascending=False)
most_popular_items['Prices'] = purchase_data.groupby(purchase_data['Item Name']).aggregate({'Price':'unique'})
most_popular_items.head(5)

,Purchase Count,Item ID,Total Purchase Value,Prices
Item Name,,,,
Final Critic,13,"[92, 101]",59.99,"[4.88, 4.19]"
"Oathbreaker, Last Hope of the Breaking Storm",12,[178],50.76,[4.23]
Persuasion,9,"[141, 132]",28.99,"[3.19, 3.33]"
Nirvana,9,[82],44.10,[4.9]
"Extraction, Quickblade Of Trembling Hands",9,[108],31.77,[3.53]


In [90]:
# ### Most Profitable Items

# * Identify the 5 most profitable items by total purchase value, then list (in a table):
#   * Item ID
#   * Item Name
#   * Purchase Count
#   * Item Price
#   * Total Purchase Value


aggregation_functions = {'Purchase ID':'count','Item ID':'unique','Price':'sum'}
most_popular_items = purchase_data.groupby(purchase_data['Item Name']).aggregate(aggregation_functions)
most_popular_items = most_popular_items.rename(columns={'Purchase ID': 'Purchase Count','Price': 'Total Purchase Value'})
most_popular_items = most_popular_items.sort_values(by='Total Purchase Value',ascending=False)
most_popular_items['Prices'] = purchase_data.groupby(purchase_data['Item Name']).aggregate({'Price':'unique'})
most_popular_items.head(5)

,Purchase Count,Item ID,Total Purchase Value,Prices
Item Name,,,,
Final Critic,13,"[92, 101]",59.99,"[4.88, 4.19]"
"Oathbreaker, Last Hope of the Breaking Storm",12,[178],50.76,[4.23]
Nirvana,9,[82],44.10,[4.9]
Fiery Glass Crusader,9,[145],41.22,[4.58]
Singed Scalpel,8,[103],34.80,[4.35]


In [ ]:

# total_value = []
# indexes = []
# for row in most_popular_items.index: 

#     total_value.append(most_popular_items['Price'][row].sum())
#     indexes.append(row)

# most_popular_items['Total Purchase Value'] = pd.Series(total_value, index = indexes) 